Aula 1 -Tabela LL(1)

Célula 1 — Gramática + FIRST e FOLLOW

In [1]:
#@title Gramática (Expr/Term/Factor) + FIRST e FOLLOW
from collections import defaultdict

EPS = 'ε'  # epsilon

# Gramática sem recursão à esquerda (mesma usada nas aulas)
G = {
    'E' : [["T","E'"]],
    "E'": [["+", "T", "E'"], ["-", "T", "E'"], [EPS]],
    'T' : [["F","T'"]],
    "T'": [["*", "F", "T'"], ["/", "F", "T'"], [EPS]],
    'F' : [["(", "E", ")"], ["num"], ["id"]],
}
NONTERMS = set(G.keys())

# Terminais (deduzidos da gramática) + EOF
TERMS = set()
for A, prods in G.items():
    for rhs in prods:
        for s in rhs:
            if s not in NONTERMS and s != EPS:
                TERMS.add(s)
TERMS |= {"EOF"}

def first_of_seq(seq, FIRST):
    """FIRST de uma sequência (lista) de símbolos."""
    out = set()
    if not seq:
        out.add(EPS);
        return out
    for X in seq:
        if X in NONTERMS:
            out |= (FIRST[X] - {EPS})
            if EPS in FIRST[X]:
                continue
            return out
        else:
            out.add(X)
            return out
    out.add(EPS)
    return out

# --- FIRST ---
FIRST = {A:set() for A in NONTERMS}
changed = True
while changed:
    changed = False
    for A, prods in G.items():
        for rhs in prods:
            before = len(FIRST[A])
            s = first_of_seq(rhs, FIRST)
            FIRST[A] |= s
            changed |= (len(FIRST[A]) != before)

# --- FOLLOW ---
FOLLOW = {A:set() for A in NONTERMS}
FOLLOW['E'].add("EOF")  # símbolo inicial

changed = True
while changed:
    changed = False
    for A, prods in G.items():
        for rhs in prods:
            # varrer rhs e aplicar as regras de FOLLOW
            for i, X in enumerate(rhs):
                if X in NONTERMS:
                    beta = rhs[i+1:]
                    fbeta = first_of_seq(beta, FIRST)
                    before = len(FOLLOW[X])
                    FOLLOW[X] |= (fbeta - {EPS})
                    if (not beta) or (EPS in fbeta):
                        FOLLOW[X] |= FOLLOW[A]
                    changed |= (len(FOLLOW[X]) != before)

print("FIRST:")
for A in sorted(NONTERMS):
    print(f"  FIRST({A}) =", sorted(FIRST[A]))

print("\nFOLLOW:")
for A in sorted(NONTERMS):
    print(f"  FOLLOW({A}) =", sorted(FOLLOW[A]))


FIRST:
  FIRST(E) = ['(', 'id', 'num']
  FIRST(E') = ['+', '-', 'ε']
  FIRST(F) = ['(', 'id', 'num']
  FIRST(T) = ['(', 'id', 'num']
  FIRST(T') = ['*', '/', 'ε']

FOLLOW:
  FOLLOW(E) = [')', 'EOF']
  FOLLOW(E') = [')', 'EOF']
  FOLLOW(F) = [')', '*', '+', '-', '/', 'EOF']
  FOLLOW(T) = [')', '+', '-', 'EOF']
  FOLLOW(T') = [')', '+', '-', 'EOF']


Célula 2 — Construção da Tabela LL(1) + checagens

In [2]:
#@title Tabela LL(1) + validação (conflitos e lacunas relevantes)
# Monta a tabela M[nao_terminal][terminal] -> produção (lista)
M = {A:{t:None for t in sorted(TERMS)} for A in NONTERMS}
conflicts = []  # (A, terminal, prod_existente, prod_nova)

def add_entry(A, terminal, rhs):
    cur = M[A][terminal]
    if cur is None:
        M[A][terminal] = rhs
    elif cur != rhs:
        conflicts.append((A, terminal, cur, rhs))

# Regras de preenchimento
for A, prods in G.items():
    for rhs in prods:
        f = first_of_seq(rhs, FIRST)
        for a in (f - {EPS}):
            add_entry(A, a, rhs)
        if EPS in f:
            for b in FOLLOW[A]:
                add_entry(A, b, rhs)

# "Lacunas relevantes": colunas esperadas para A (FIRSTs e FOLLOW se tiver ε)
missing = []
for A, prods in G.items():
    expected_cols = set()
    for rhs in prods:
        f = first_of_seq(rhs, FIRST)
        expected_cols |= (f - {EPS})
        if EPS in f:
            expected_cols |= FOLLOW[A]
    for t in sorted(expected_cols):
        if M[A][t] is None:
            missing.append((A, t))

# Impressão compacta da tabela
cols = ['(', ')', 'id', 'num', '+', '-', '*', '/', 'EOF']
cols = [c for c in cols if c in TERMS]  # só os disponíveis
w = 12
print(" "*(w), *[c.center(w) for c in cols])
for A in sorted(NONTERMS):
    row = [A.ljust(w)]
    for c in cols:
        rhs = M[A][c]
        cell = "" if rhs is None else (A+"→"+" ".join(rhs))
        row.append(cell.center(w))
    print("".join(row))

print("\nConflitos:", "nenhum" if not conflicts else len(conflicts))
if conflicts:
    for A, t, p1, p2 in conflicts:
        print(f"  [{A}, {t}] : {A}→{' '.join(p1)}  x  {A}→{' '.join(p2)}")

print("\nLacunas relevantes:", "nenhuma" if not missing else len(missing))
if missing:
    for A, t in missing:
        print(f"  faltando ação para {A} com lookahead {t}")


                  (            )            id          num           +            -            *            /           EOF     
E              E→T E'                  E→T E'      E→T E'                                                               
E'                          E'→ε                             E'→+ T E'   E'→- T E'                              E'→ε    
F             F→( E )                   F→id       F→num                                                                
T              T→F T'                  T→F T'      T→F T'                                                               
T'                          T'→ε                                T'→ε        T'→ε     T'→* F T'   T'→/ F T'      T'→ε    

Conflitos: nenhum

Lacunas relevantes: nenhuma


Célula 3 — Usando a tabela para reconhecer entradas (aceita/erro)

In [3]:
#@title Reconhecedor LL(1) usando a tabela (com exemplos)
import re

def lex(src: str):
    toks = []
    i=0
    while i < len(src):
        ch = src[i]
        if ch.isspace():
            i+=1; continue
        if ch.isdigit():
            j=i
            while j<len(src) and src[j].isdigit(): j+=1
            toks.append(("num", src[i:j]))
            i=j; continue
        if ch.isalpha() or ch=='_':
            j=i
            while j<len(src) and (src[j].isalnum() or src[j]=='_'): j+=1
            toks.append(("id", src[i:j]))
            i=j; continue
        if ch in "+-*/()":
            toks.append((ch,ch)); i+=1; continue
        raise ValueError(f"Caractere inválido: {ch!r} na pos {i}")
    toks.append(("EOF",""))
    return [k for (k,_) in toks]  # devolve só os tipos/lexemas normalizados

def parse_ll1(src: str, verbose=True):
    tokens = lex(src)
    stack = ["EOF", "E"]  # pilha começa com símbolo inicial no topo
    i = 0  # índice em tokens

    def is_nonterm(x): return x in NONTERMS
    if verbose:
        print(f"Tokens: {' '.join(tokens)}")

    step = 0
    while stack:
        top = stack.pop()
        look = tokens[i]
        if verbose:
            print(f"{step:02d}  topo={top:>3}   lookahead={look:>4}   pilha={stack}")

        if top in TERMS:  # terminal
            if top == look:
                i += 1  # consome
            else:
                exp = top
                print(f"ERRO: esperado `{exp}`, veio `{look}`.")
                return False
        elif is_nonterm(top):
            rhs = M[top].get(look)
            if rhs is None:
                # mostra opções válidas para esse não-terminal
                opts = [t for t in TERMS if M[top].get(t)]
                print(f"ERRO: em `{top}` com `{look}`. Esperado um de {opts}.")
                return False
            # empilha produção ao contrário (ignorando ε)
            for s in reversed(rhs):
                if s != EPS:
                    stack.append(s)
        else:
            print(f"Símbolo desconhecido na pilha: {top}")
            return False
        step += 1

    ok = (i == len(tokens))
    print("ACEITA" if ok else "ERRO: tokens restantes.")
    return ok

# Exemplos (saída esperada comentada)
tests = [
    "2 + 3 * 4",      # ACEITA
    "(2 + 3) * 4",    # ACEITA
    "x * (y + 1) - 5",# ACEITA
    "2 + * 3",        # ERRO (operando faltando)
    "(2 + 3 * 4"      # ERRO (faltou ')')
]

for s in tests:
    print("="*60)
    print("Entrada:", s)
    parse_ll1(s, verbose=False)


Entrada: 2 + 3 * 4
ACEITA
Entrada: (2 + 3) * 4
ACEITA
Entrada: x * (y + 1) - 5
ACEITA
Entrada: 2 + * 3
ERRO: em `T` com `*`. Esperado um de ['(', 'num', 'id'].
Entrada: (2 + 3 * 4
ERRO: esperado `)`, veio `EOF`.


Aula 2 - Parser recursivo com Python no Colab

In [4]:
#@title Parser recursivo (Expr/Term/Factor) + AST + exemplos
import re
from dataclasses import dataclass

# --------- Lexer simples ---------
@dataclass
class Token:
    kind: str   # 'NUM','ID','PLUS','MINUS','MUL','DIV','LPAREN','RPAREN','EOF'
    lexeme: str
    pos: int

_spec = [
    ('NUM',    r'\d+'),
    ('ID',     r'[a-zA-Z_][a-zA-Z0-9_]*'),
    ('PLUS',   r'\+'),
    ('MINUS',  r'-'),
    ('MUL',    r'\*'),
    ('DIV',    r'/'),
    ('LPAREN', r'\('),
    ('RPAREN', r'\)'),
    ('WS',     r'[ \t\n\r]+'),
]
_tok = re.compile('|'.join(f'(?P<{k}>{p})' for k,p in _spec))

def lex(src: str):
    out = []
    for m in _tok.finditer(src):
        k = m.lastgroup
        if k == 'WS':
            continue
        out.append(Token(k, m.group(), m.start()))
    out.append(Token('EOF','', len(src)))
    return out

def show_error(src: str, pos: int, msg: str):
    line_start = src.rfind('\n', 0, pos) + 1
    line_end   = src.find('\n', pos)
    if line_end == -1: line_end = len(src)
    line = src[line_start:line_end]
    col = pos - line_start
    print(f"Erro de sintaxe na coluna {col+1}: {msg}")
    print(line)
    print(' ' * col + '^')

# --------- AST ---------
@dataclass
class AST: ...
@dataclass
class Num(AST): value: int
@dataclass
class Id(AST):  name: str
@dataclass
class Bin(AST): op: str; left: AST; right: AST

def to_infix(ast: AST) -> str:
    if isinstance(ast, Num): return str(ast.value)
    if isinstance(ast, Id):  return ast.name
    if isinstance(ast, Bin): return f"({to_infix(ast.left)} {ast.op} {to_infix(ast.right)})"
    return "<??>"

# --------- Parser recursivo-descendente ---------
class ParseError(Exception): ...

class Parser:
    """
    Gramática usada:
      Expr   → Term (('+'|'-') Term)*
      Term   → Factor (('*'|'/') Factor)*
      Factor → '(' Expr ')' | NUM | ID
    """
    def __init__(self, src: str):
        self.src = src
        self.toks = lex(src)
        self.i = 0

    def peek(self): return self.toks[self.i]
    def eat(self, kind: str):
        t = self.peek()
        if t.kind != kind:
            raise ParseError((t.pos, f"esperado {kind}; veio {t.kind}"))
        self.i += 1
        return t

    def parse(self) -> AST:
        node = self.parse_expr()
        self.eat('EOF')
        return node

    # Expr -> Term (('+'|'-') Term)*
    def parse_expr(self) -> AST:
        left = self.parse_term()
        while self.peek().kind in ('PLUS','MINUS'):
            k = self.eat(self.peek().kind).kind
            op = '+' if k=='PLUS' else '-'
            right = self.parse_term()
            left = Bin(op, left, right)
        return left

    # Term -> Factor (('*'|'/') Factor)*
    def parse_term(self) -> AST:
        left = self.parse_factor()
        while self.peek().kind in ('MUL','DIV'):
            k = self.eat(self.peek().kind).kind
            op = '*' if k=='MUL' else '/'
            right = self.parse_factor()
            left = Bin(op, left, right)
        return left

    # Factor -> '(' Expr ')' | NUM | ID
    def parse_factor(self) -> AST:
        t = self.peek()
        if t.kind == 'LPAREN':
            self.eat('LPAREN')
            node = self.parse_expr()
            if self.peek().kind != 'RPAREN':
                raise ParseError((self.peek().pos, "esperado RPAREN; veio " + self.peek().kind))
            self.eat('RPAREN')
            return node
        if t.kind == 'NUM':
            n = self.eat('NUM');  return Num(int(n.lexeme))
        if t.kind == 'ID':
            n = self.eat('ID');   return Id(n.lexeme)
        raise ParseError((t.pos, "token inesperado em Factor: " + t.kind))

def parse_or_error(src: str):
    p = Parser(src)
    try:
        ast = p.parse()
        print("AST:", ast)
        print("Infixo:", to_infix(ast))
    except ParseError as e:
        pos, msg = e.args[0]
        show_error(src, pos, msg)

# --------- Exemplos rápidos ---------
tests = [
    "2 + 3 * 4",        # válido → (2 + (3 * 4))
    "(2 + 3) * 4",      # válido → ((2 + 3) * 4)
    "x * (y + 1) - 5",  # válido → ((x * (y + 1)) - 5)
    "2 + * 3",          # inválido (operando faltando)
    "(2 + 3 * 4"        # inválido (faltou ')')
]
for s in tests:
    print("="*60)
    print("Entrada:", s)
    parse_or_error(s)


Entrada: 2 + 3 * 4
AST: Bin(op='+', left=Num(value=2), right=Bin(op='*', left=Num(value=3), right=Num(value=4)))
Infixo: (2 + (3 * 4))
Entrada: (2 + 3) * 4
AST: Bin(op='*', left=Bin(op='+', left=Num(value=2), right=Num(value=3)), right=Num(value=4))
Infixo: ((2 + 3) * 4)
Entrada: x * (y + 1) - 5
AST: Bin(op='-', left=Bin(op='*', left=Id(name='x'), right=Bin(op='+', left=Id(name='y'), right=Num(value=1))), right=Num(value=5))
Infixo: ((x * (y + 1)) - 5)
Entrada: 2 + * 3
Erro de sintaxe na coluna 5: token inesperado em Factor: MUL
2 + * 3
    ^
Entrada: (2 + 3 * 4
Erro de sintaxe na coluna 11: esperado RPAREN; veio EOF
(2 + 3 * 4
          ^


Aula 3 -Analisando ambiguidade gramatical na prática

In [5]:
#@title Analisando ambiguidade na prática (uma célula)
# Gramática ambígua (intuitiva, mas problemática):
#   E → E '+' E | E '*' E | num
# Não há precedência nem associatividade definidas, então
# a mesma entrada pode ter MAIS DE UMA árvore.

from dataclasses import dataclass

# ---------- AST mínima ----------
@dataclass
class Num:
    v: int
@dataclass
class Bin:
    op: str
    left: object
    right: object

def to_infix(node):
    if isinstance(node, Num): return str(node.v)
    if isinstance(node, Bin): return f"({to_infix(node.left)} {node.op} {to_infix(node.right)})"
    return "<?>"

# ---------- Lexer simples (nums, +, *) ----------
def lex(src: str):
    toks = []
    i = 0
    while i < len(src):
        ch = src[i]
        if ch.isspace():
            i += 1; continue
        if ch.isdigit():
            j = i
            while j < len(src) and src[j].isdigit(): j += 1
            toks.append(("NUM", int(src[i:j])))
            i = j; continue
        if ch in "+*":
            toks.append((ch, ch)); i += 1; continue
        raise ValueError(f"Caractere inválido: {ch!r} na posição {i}")
    return toks

# ---------- Geração de TODAS as árvores possíveis ----------
# Entrada esperada: sequência alternando NUM op NUM op NUM ...
# Estratégia: para cada operador na faixa, divide em (esq op dir) e combina
# todas as árvores possíveis da esquerda com as da direita.
from functools import lru_cache

@lru_cache(None)
def parse_all(tokens_tuple, i, j):
    # tokens_tuple: tupla de pares (tipo,valor); i..j inclusive
    # Base: um único número
    if i == j:
        typ, val = tokens_tuple[i]
        if typ == "NUM": return [Num(val)]
        return []
    trees = []
    # Particiona apenas em posições de operador (índices ímpares no padrão NUM op NUM ...)
    for k in range(i+1, j, 2):
        op_typ, op_val = tokens_tuple[k]
        if op_typ not in ['+','*']:
            continue
        left_trees  = parse_all(tokens_tuple, i, k-1)
        right_trees = parse_all(tokens_tuple, k+1, j)
        for L in left_trees:
            for R in right_trees:
                trees.append(Bin(op_val, L, R))
    return trees

def analisar(src: str):
    print("="*60)
    print("Entrada:", src)
    toks = lex(src)
    # validação simples: precisa ser NUM (op NUM)*
    if not toks or toks[0][0] != "NUM" or any(toks[t][0] == toks[t+1][0] for t in range(len(toks)-1)):
        print("Formato inválido para este exemplo (use: num (+|*) num ...).")
        return
    trees = parse_all(tuple(toks), 0, len(toks)-1)
    # Remover duplicatas (mesma string infixa) se houver
    uniq = {}
    for t in trees:
        s = to_infix(t)
        uniq.setdefault(s, t)
    formas = list(uniq.keys())
    formas.sort()
    for i, s in enumerate(formas, 1):
        print(f"Árvore {i:02d}:", s)
    if len(formas) > 1:
        print(f"\n→ AMBÍGUA: {len(formas)} árvores diferentes para a MESMA entrada.")
        print("  (Não há precedência/associatividade definidas.)")
    else:
        print("\n→ NÃO ambígua nesta entrada (só uma árvore).")

# ---------- Demonstrações ----------
analisar("2 + 3 * 4")       # clássico: duas árvores -> ambígua
analisar("2 + 3 + 4")       # duas árvores por associatividade
analisar("7 * 8")           # apenas uma árvore


Entrada: 2 + 3 * 4
Árvore 01: ((2 + 3) * 4)
Árvore 02: (2 + (3 * 4))

→ AMBÍGUA: 2 árvores diferentes para a MESMA entrada.
  (Não há precedência/associatividade definidas.)
Entrada: 2 + 3 + 4
Árvore 01: ((2 + 3) + 4)
Árvore 02: (2 + (3 + 4))

→ AMBÍGUA: 2 árvores diferentes para a MESMA entrada.
  (Não há precedência/associatividade definidas.)
Entrada: 7 * 8
Árvore 01: (7 * 8)

→ NÃO ambígua nesta entrada (só uma árvore).


Aula 4 - Parser LL(1) com backtracking

In [6]:
#@title Parser LL(1) com backtracking limitado em Stmt (1 célula)
import re
from dataclasses import dataclass

# ----------------- Lexer -----------------
@dataclass
class Token:
    kind: str   # 'ID','NUM','LP','RP','COMMA','EQ','PLUS','MINUS','MUL','DIV','EOF'
    lexeme: str
    pos: int

_tokspec = [
    ('NUM',   r'\d+'),
    ('ID',    r'[a-zA-Z_][a-zA-Z0-9_]*'),
    ('EQ',    r'='),
    ('LP',    r'\('),
    ('RP',    r'\)'),
    ('COMMA', r','),
    ('PLUS',  r'\+'),
    ('MINUS', r'-'),
    ('MUL',   r'\*'),
    ('DIV',   r'/'),
    ('WS',    r'[ \t\n\r]+'),
]
_tokre = re.compile('|'.join(f'(?P<{k}>{p})' for k,p in _tokspec))

def lex(src: str):
    toks=[]
    for m in _tokre.finditer(src):
        k=m.lastgroup
        if k=='WS': continue
        toks.append(Token(k, m.group(), m.start()))
    toks.append(Token('EOF','', len(src)))
    return toks

def show_error(src: str, pos: int, msg: str):
    line_start = src.rfind('\n', 0, pos) + 1
    line_end = src.find('\n', pos)
    if line_end == -1: line_end = len(src)
    line = src[line_start:line_end]
    col = pos - line_start
    print(f"Erro: {msg} (coluna {col+1})")
    print(line)
    print(" " * col + "^")

# ----------------- AST -----------------
@dataclass
class AST: ...
@dataclass
class Num(AST): value: int
@dataclass
class Id(AST):  name: str
@dataclass
class Bin(AST): op: str; left: AST; right: AST
@dataclass
class Assign(AST): name: str; expr: AST
@dataclass
class Call(AST):   name: str; args: list

def to_infix(e: AST) -> str:
    if isinstance(e, Num): return str(e.value)
    if isinstance(e, Id):  return e.name
    if isinstance(e, Bin): return f"({to_infix(e.left)} {e.op} {to_infix(e.right)})"
    return "<?>"

# ----------------- Parser -----------------
class ParseError(Exception): ...

class Parser:
    """
    Stmt → ID '=' Expr | ID '(' Args ')'
    Expr → Term (('+'|'-') Term)*
    Term → Factor (('*'|'/') Factor)*
    Factor → '(' Expr ')' | NUM | ID
    Args → Expr (',' Expr)* | ε
    """
    def __init__(self, src: str):
        self.src = src
        self.toks = lex(src)
        self.i = 0

    def peek(self): return self.toks[self.i]
    def eat(self, kind: str):
        t = self.peek()
        if t.kind != kind:
            raise ParseError((t.pos, f"esperado {kind}; veio {t.kind}"))
        self.i += 1
        return t

    # -------- Stmt com backtracking limitado --------
    def parse_stmt(self) -> AST:
        # salva posição
        save = self.i
        try:
            # Tente: ID '=' Expr
            name = self.eat('ID').lexeme          # pode avançar um ID sem medo
            self.eat('EQ')                        # falha cedo se não for '='
            expr = self.parse_expr()
            self.eat('EOF')
            return Assign(name, expr)
        except ParseError:
            # restaura e tenta: ID '(' Args ')'
            self.i = save
            try:
                name = self.eat('ID').lexeme
                self.eat('LP')
                args = self.parse_args()
                self.eat('RP')
                self.eat('EOF')
                return Call(name, args)
            except ParseError as e2:
                # erro final: mostre o que poderia vir após ID
                pos, _ = e2.args[0]
                exp = "esperado '=' ou '(' após ID"
                raise ParseError((pos, exp))

    # -------- Args --------
    def parse_args(self):
        # ε
        if self.peek().kind == 'RP':
            return []
        # Expr (',' Expr)*
        args = [self.parse_expr()]
        while self.peek().kind == 'COMMA':
            self.eat('COMMA')
            args.append(self.parse_expr())
        return args

    # -------- Expr / Term / Factor --------
    def parse_expr(self) -> AST:
        left = self.parse_term()
        while self.peek().kind in ('PLUS','MINUS'):
            k = self.eat(self.peek().kind).kind
            op = '+' if k=='PLUS' else '-'
            right = self.parse_term()
            left = Bin(op, left, right)
        return left

    def parse_term(self) -> AST:
        left = self.parse_factor()
        while self.peek().kind in ('MUL','DIV'):
            k = self.eat(self.peek().kind).kind
            op = '*' if k=='MUL' else '/'
            right = self.parse_factor()
            left = Bin(op, left, right)
        return left

    def parse_factor(self) -> AST:
        t = self.peek()
        if t.kind == 'LP':
            self.eat('LP')
            node = self.parse_expr()
            if self.peek().kind != 'RP':
                raise ParseError((self.peek().pos, "esperado RP; veio " + self.peek().kind))
            self.eat('RP')
            return node
        if t.kind == 'NUM':
            n = self.eat('NUM'); return Num(int(n.lexeme))
        if t.kind == 'ID':
            n = self.eat('ID');  return Id(n.lexeme)
        raise ParseError((t.pos, "token inesperado em Factor: " + t.kind))

# --------------- Runner ---------------
def run(src: str):
    print("="*70)
    print("Entrada:", src)
    p = Parser(src)
    try:
        node = p.parse_stmt()
        if isinstance(node, Assign):
            print("OK: ASSIGN")
            print("AST:", node)
            print("Expr infixo:", to_infix(node.expr))
        elif isinstance(node, Call):
            print("OK: CALL")
            print("AST:", node)
            print("Args infixos:", [to_infix(a) for a in node.args])
        else:
            print("OK:", node)
    except ParseError as e:
        pos, msg = e.args[0]
        show_error(src, pos, msg)

# --------------- Demonstrações ---------------
tests = [
    "x = 1 + 2*3",         # ASSIGN
    "f(x, 2)",             # CALL
    "a(b + c, 3*4)",       # CALL
    "a",                   # ERRO: esperado '=' ou '(' após ID
    "g(,1)",               # ERRO: vírgula sem expressão
    "h * 2"                # ERRO: não é Stmt (nem assign nem call)
]
for t in tests:
    run(t)


Entrada: x = 1 + 2*3
OK: ASSIGN
AST: Assign(name='x', expr=Bin(op='+', left=Num(value=1), right=Bin(op='*', left=Num(value=2), right=Num(value=3))))
Expr infixo: (1 + (2 * 3))
Entrada: f(x, 2)
OK: CALL
AST: Call(name='f', args=[Id(name='x'), Num(value=2)])
Args infixos: ['x', '2']
Entrada: a(b + c, 3*4)
OK: CALL
AST: Call(name='a', args=[Bin(op='+', left=Id(name='b'), right=Id(name='c')), Bin(op='*', left=Num(value=3), right=Num(value=4))])
Args infixos: ['(b + c)', '(3 * 4)']
Entrada: a
Erro: esperado '=' ou '(' após ID (coluna 2)
a
 ^
Entrada: g(,1)
Erro: esperado '=' ou '(' após ID (coluna 3)
g(,1)
  ^
Entrada: h * 2
Erro: esperado '=' ou '(' após ID (coluna 3)
h * 2
  ^


Aula 5 - Testes de sintaxe com diferentes entradas

In [7]:
#@title Testes de sintaxe: parser recursivo + AST + PASS/FAIL (1 célula)
import re
from dataclasses import dataclass

# ============ Lexer ============

@dataclass
class Token:
    kind: str   # 'NUM','ID','PLUS','MINUS','MUL','DIV','LPAREN','RPAREN','EOF'
    lexeme: str
    pos: int

_spec = [
    ('NUM',    r'\d+'),
    ('ID',     r'[a-zA-Z_][a-zA-Z0-9_]*'),
    ('PLUS',   r'\+'),
    ('MINUS',  r'-'),
    ('MUL',    r'\*'),
    ('DIV',    r'/'),
    ('LPAREN', r'\('),
    ('RPAREN', r'\)'),
    ('WS',     r'[ \t\n\r]+'),
]
_tok = re.compile('|'.join(f'(?P<{k}>{p})' for k,p in _spec))

def lex(src: str):
    out = []
    for m in _tok.finditer(src):
        k = m.lastgroup
        if k == 'WS':
            continue
        out.append(Token(k, m.group(), m.start()))
    out.append(Token('EOF','', len(src)))
    return out

def mostra_erro(src: str, pos: int, msg: str):
    ini = src.rfind('\n', 0, pos) + 1
    fim = src.find('\n', pos)
    if fim == -1: fim = len(src)
    linha = src[ini:fim]
    col = pos - ini
    print(f"Erro de sintaxe na coluna {col+1}: {msg}")
    print(linha)
    print(' ' * col + '^')

# ============ AST ============

class AST: ...
@dataclass
class Num(AST): value: int
@dataclass
class Id(AST):  name: str
@dataclass
class Bin(AST): op: str; left: AST; right: AST

def to_infix(ast: AST) -> str:
    if isinstance(ast, Num): return str(ast.value)
    if isinstance(ast, Id):  return ast.name
    if isinstance(ast, Bin): return f"({to_infix(ast.left)} {ast.op} {to_infix(ast.right)})"
    return "<?>"

# ============ Parser recursivo (Expr/Term/Factor) ============

class ParseError(Exception): ...

class Parser:
    """
    Gramática (sem recursão à esquerda):
      Expr   → Term (('+'|'-') Term)*
      Term   → Factor (('*'|'/') Factor)*
      Factor → '(' Expr ')' | NUM | ID
    """
    def __init__(self, src: str):
        self.src = src
        self.toks = lex(src)
        self.i = 0

    def peek(self): return self.toks[self.i]
    def eat(self, kind: str):
        t = self.peek()
        if t.kind != kind:
            raise ParseError((t.pos, f"esperado {kind}; veio {t.kind}"))
        self.i += 1
        return t

    def parse(self) -> AST:
        node = self.parse_expr()
        self.eat('EOF')
        return node

    def parse_expr(self) -> AST:
        left = self.parse_term()
        while self.peek().kind in ('PLUS','MINUS'):
            k = self.eat(self.peek().kind).kind
            op = '+' if k=='PLUS' else '-'
            right = self.parse_term()
            left = Bin(op, left, right)
        return left

    def parse_term(self) -> AST:
        left = self.parse_factor()
        while self.peek().kind in ('MUL','DIV'):
            k = self.eat(self.peek().kind).kind
            op = '*' if k=='MUL' else '/'
            right = self.parse_factor()
            left = Bin(op, left, right)
        return left

    def parse_factor(self) -> AST:
        t = self.peek()
        if t.kind == 'LPAREN':
            self.eat('LPAREN')
            node = self.parse_expr()
            if self.peek().kind != 'RPAREN':
                raise ParseError((self.peek().pos, "esperado RPAREN; veio " + self.peek().kind))
            self.eat('RPAREN')
            return node
        if t.kind == 'NUM':
            n = self.eat('NUM');  return Num(int(n.lexeme))
        if t.kind == 'ID':
            n = self.eat('ID');   return Id(n.lexeme)
        raise ParseError((t.pos, "token inesperado em Factor: " + t.kind))

# ============ Suite de testes (válidos/ inválidos) ============

validos = [
    ("2 + 3 * 4",           "(2 + (3 * 4))"),
    ("(2 + 3) * 4",         "((2 + 3) * 4)"),
    ("x * (y + 1) - 5",     "((x * (y + 1)) - 5)"),
    ("a1 + _x2",            "(a1 + _x2)"),
    ("((7))",               "7"),
]

invalidos = [
    ("2 + * 3",        "esperado NUM"),        # operando faltando
    ("(2 + 3 * 4",     "esperado RPAREN"),     # faltou ')'
    ("a + + b",        "token inesperado"),    # operador duplo
    ("* 3",            "token inesperado"),    # início inválido
]

# ============ Runner ============

def testa_valido(src, esperado):
    try:
        ast = Parser(src).parse()
        got = to_infix(ast)
        ok = (got == esperado)
        print(("PASS" if ok else "FAIL"), "|", src, "|", got)
        if not ok:
            print("  esperado:", esperado)
        return ok
    except ParseError as e:
        pos, msg = e.args[0]
        print("FAIL |", src, "| erro inesperado:")
        mostra_erro(src, pos, msg)
        return False

def testa_invalido(src, trecho_msg):
    try:
        Parser(src).parse()
        print("FAIL |", src, "| deveria falhar, mas aceitou.")
        return False
    except ParseError as e:
        pos, msg = e.args[0]
        ok = (trecho_msg in msg)
        print(("PASS" if ok else "FAIL"), "|", src, "|", msg)
        if not ok:
            print("  esperado conter:", trecho_msg)
        return ok

ok_v = sum(testa_valido(s, exp) for s,exp in validos)
ok_i = sum(testa_invalido(s, exp) for s,exp in invalidos)
tot_v, tot_i = len(validos), len(invalidos)

print("\nResumo:")
print(f"  Válidos:   {ok_v}/{tot_v} PASS")
print(f"  Inválidos: {ok_i}/{tot_i} PASS")
print(f"  TOTAL:     {ok_v+ok_i}/{tot_v+tot_i} PASS")

# (Opcional) teste livre rápido:
def testar_rapido(expr: str):
    print("\nTeste livre:", expr)
    try:
        ast = Parser(expr).parse()
        print("AST:", ast)
        print("Infixo:", to_infix(ast))
    except ParseError as e:
        pos, msg = e.args[0]
        mostra_erro(expr, pos, msg)

# Exemplo de uso:
# testar_rapido("2 + (3 * 4)")

PASS | 2 + 3 * 4 | (2 + (3 * 4))
PASS | (2 + 3) * 4 | ((2 + 3) * 4)
PASS | x * (y + 1) - 5 | ((x * (y + 1)) - 5)
PASS | a1 + _x2 | (a1 + _x2)
PASS | ((7)) | 7
FAIL | 2 + * 3 | token inesperado em Factor: MUL
  esperado conter: esperado NUM
PASS | (2 + 3 * 4 | esperado RPAREN; veio EOF
PASS | a + + b | token inesperado em Factor: PLUS
PASS | * 3 | token inesperado em Factor: MUL

Resumo:
  Válidos:   5/5 PASS
  Inválidos: 3/4 PASS
  TOTAL:     8/9 PASS
